In [ ]:
import math
import random
import csv
import sys
import operator
import pickle
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from shapely.geometry import Polygon, Point

import cvxpy as cp

import keras
from keras import layers
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.models import Model
from keras import backend as K
from keras import metrics, Sequential
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy.special import comb, perm
import time

In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

## Mount Google Drive Folder

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')
rootPath = '/content/drive/Shareddrives/KW phase1 repo'

Mounted at /content/drive


In [ ]:
import sys
sys.path.append(rootPath+'/package')
from utils.env_sdp import env
from model.uav import UAV

In [ ]:
deviations = [1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1]
GetDevicationStr = lambda deviation: str(int(deviation*100))
dict_slice = lambda adict, start, end: { k:adict[k] for k in list (adict.keys())[start:end] }

## AE 

In [ ]:
from model.uav import UAV, setUAVEnv
from model.pureSDP import calculateSDP, setSDPEnv
from model.pathlossmodel import PathLossModel, NoiseMaker, setPathLossModelEnv

from service.distributeuavs import createUAVsSamples,setDistributeUavsEnv , TestEnv
from service.measurerss import measureRSS, setMeasuerRssEnv
from service.combinationset import combinationSet , setCombinationSetEnv
from multiprocessing import Pool
from sklearn.manifold import TSNE

setDistributeUavsEnv("sdp")
TestEnv()
setCombinationSetEnv("sdp")
setPathLossModelEnv("sdp")
setMeasuerRssEnv("sdp")
setUAVEnv("sdp")
setSDPEnv("sdp")

envName sdp
16


In [ ]:
batch_size = env.batch_size*10
latent_dim = env.latent_dim
intermediate_dim =env.intermediate_dim
final_dim = env.final_dim
epochs = env.epochs//5
epsilon_std = env.epsilon_std

In [ ]:
class Autoencoder(Model):
  def __init__(self, inputSize):
    super().__init__()
    
    self.encoder = Sequential(
        [
         Flatten(),
         Dense(intermediate_dim, activation='relu'),
         Dense(final_dim, activation='relu')
        ]
    )

    self.decoder = Sequential(
        [
         Dense(final_dim, activation='relu'),
         Dense(intermediate_dim, activation='relu'),
         Dense(inputSize, activation='sigmoid')
        ]
    )

  def call(self, inputs):
    encoded = self.encoder(inputs)
    decoded = self.decoder(encoded)
    return decoded
  
  def encode(self, inputs):
    return self.encoder(inputs)

In [ ]:
def dualFunc(X,i,j): 
    temp = []
    for index_1, e1 in enumerate(X[i][j]):
      for index_2, e2 in enumerate(X[i][j]):
        if not index_1 == index_2:
          temp.append([e1, e2])
    return temp

In [ ]:
def swapFunc(j,X,shapeOfX,targetReformSlice): 
  for sample in range(shapeOfX[1]): #sample
    temp = []
    for i in range(shapeOfX[0]):
      temp.append(X[i][sample][j])
    targetReformSlice.append(temp)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
for deviation in deviations:
  _pathLossModel = PathLossModel()
  _pathLossModel.noiseMaker = NoiseMaker(mean = env.noiseMean, deviation = deviation) 
  print("Deviation【{}】".format(deviation))
  with open(rootPath+"/datas/uavPosition/300_0706/done_deviation/"+GetDevicationStr(deviation)+"/UAVsSamples.pkl", "rb") as f:
    UAVsSamples = pickle.load(f)
  UAVsSamples = dict_slice(UAVsSamples, 0, 100)

  measureRSS(UAVsSamples, _pathLossModel)

  trainingRawData = []

  for iSample, UAVSample in UAVsSamples.items():
    uavAnchorOfSegments = []
    uavTargetOfSegments = []
    uavAnchorOfSegmentsAE = []
    uavTargetOfSegmentsAE = []
    groupSet = combinationSet({0:UAVSample})[0]

    ## for GPS data
    for anchorSize, segments in groupSet.items():
      for segment in segments:
        uavAnchors = []
        for anchorID in segment:
          uavAnchors.append(UAVSample[anchorID].observedPosition)
        for i in range(env.N_gps-len(uavAnchors)):
          uavAnchors.append([0.0,0.0,0.0])
        uavAnchorOfSegments.append(uavAnchors)
    # print("GPS data shape",np.array(uavAnchorOfSegments).shape)

    ## for RSS data
    for anchorSize, segments in groupSet.items():
      for segment in segments:
        uavTargets = []
        for i, uav_i in UAVSample.items():
          if uav_i.observedPosition == None:
            temp = []
            for j, uav_j in UAVSample.items():
              if not i == j:
                rss = uav_i.searchRSSById(uav_j.id)
                temp.append(rss)
            uavTargets.append(temp)
        uavTargetOfSegments.append(uavTargets)
    # print("RSS data shape", np.array(uavTargetOfSegments).shape)
    
    uavAnchorOfSegments = np.array(uavAnchorOfSegments)
    uavTargetOfSegments = np.array(uavTargetOfSegments)
    AEdata = np.append(uavAnchorOfSegments, uavTargetOfSegments, axis=2)
    trainingRawData.append(AEdata)
  trainingRawData = np.array(trainingRawData)
  trainingRawData = np.reshape(trainingRawData, (trainingRawData.shape[2], trainingRawData.shape[0], trainingRawData.shape[1], trainingRawData.shape[3]))
  # print(trainingRawData.shape)

  testingSampleSize = int(( len(UAVsSamples) * env.TEST_PERCENTAGE )//1)
  if testingSampleSize == 0:
    testingSampleSize = 1
  traningSampleSize = int(len(UAVsSamples)-testingSampleSize)
  rawTrainingData = list(map( lambda item: item[:-testingSampleSize], trainingRawData))
  rawTestingData =  list(map( lambda item: item[traningSampleSize:], trainingRawData))

  dualGroupingTrainingData = []
  dualGroupingTestingData = []
  rawTrainingDataShape = np.array(rawTrainingData).shape
  rawTestingDataShape = np.array(rawTestingData).shape
  for i in range(rawTrainingDataShape[0]):
    trainingS = list(map(lambda j: dualFunc(rawTrainingData,i,j),np.arange(0,rawTrainingDataShape[1])))
    testingS = list(map(lambda j: dualFunc(rawTestingData,i,j),np.arange(0,rawTestingDataShape[1])))
    dualGroupingTrainingData.append(trainingS)
    dualGroupingTestingData.append(testingS)

  _trainingDataShape = np.array(dualGroupingTrainingData).shape
  _testingDataShape= np.array(dualGroupingTestingData).shape
  _reshapeTrainingData = []
  _reshapeTestingData = []
   
  list(map(lambda _sample: swapFunc(_sample,dualGroupingTrainingData,_trainingDataShape,_reshapeTrainingData), range(_trainingDataShape[2])))
  list(map(lambda _sample: swapFunc(_sample,dualGroupingTestingData,_testingDataShape,_reshapeTestingData), range(_testingDataShape[2])))
  VAETrainingData = np.array(_reshapeTrainingData).reshape(len(_reshapeTrainingData), np.prod(np.array(_reshapeTrainingData).shape[1:]))
  VAETestingData = np.array(_reshapeTestingData).reshape(len(_reshapeTestingData), np.prod(np.array(_reshapeTestingData).shape[1:]))

  data_min = 0
  data_max = max(max(env.X_RANGE,env.Y_RANGE),env.Z_RANGE)
  normalizeVAETrainingData = (VAETrainingData-data_min)/(data_max - data_min)
  normalizeVAETrainingData = normalizeVAETrainingData.astype('float32')
  normalizeVAETestingData = (VAETestingData-data_min)/(data_max - data_min)
  normalizeVAETestingData = normalizeVAETestingData.astype('float32')

  X_train = normalizeVAETrainingData
  y_train = normalizeVAETestingData

  # print(np.array(X_train).shape)
  # print(np.array(y_train).shape)

  AE = Autoencoder(X_train.shape[1])
  AE.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(1e-4))
  # AE.summary()

  history = AE.fit(X_train, X_train, 
        epochs=epochs, 
        batch_size=batch_size, 
        shuffle=True,
        validation_data=(y_train, y_train),
        callbacks=[callback],
        ).history
  
  AE.save(rootPath+"/datas/trainingModel/AEModel_8UAV_8uUAV_div{}".format(deviation))

Deviation【1】
Epoch 1/20
10/10 [==============================] - 6s 525ms/step - loss: 0.6914 - val_loss: 0.6899
Epoch 2/20
10/10 [==============================] - 5s 510ms/step - loss: 0.6887 - val_loss: 0.6868
Epoch 3/20
10/10 [==============================] - 5s 512ms/step - loss: 0.6850 - val_loss: 0.6823
Epoch 4/20
10/10 [==============================] - 5s 510ms/step - loss: 0.6796 - val_loss: 0.6754
Epoch 5/20
10/10 [==============================] - 5s 505ms/step - loss: 0.6713 - val_loss: 0.6649
Epoch 6/20
10/10 [==============================] - 5s 506ms/step - loss: 0.6590 - val_loss: 0.6496
Epoch 7/20
10/10 [==============================] - 5s 506ms/step - loss: 0.6414 - val_loss: 0.6285
Epoch 8/20
10/10 [==============================] - 5s 510ms/step - loss: 0.6176 - val_loss: 0.6004
Epoch 9/20
10/10 [==============================] - 5s 507ms/step - loss: 0.5862 - val_loss: 0.5641
Epoch 10/20
10/10 [==============================] - 5s 506ms/step - loss: 0.5463 - val

In [ ]:
fig = plt.figure(figsize=(10,6))
acc = []
valAcc = []
step = 1
for i in range(len(history['loss'])//step):
  acc.append(np.mean(history['loss'][i:i+step:1]))
  valAcc.append(np.mean(history['val_loss'][i:i+step:1]))

plt.plot(acc)
plt.plot(valAcc)
plt.title('Loss (average with every {} epoch)'.format(step), fontsize=20)
plt.ylabel('loss', fontsize=18)
plt.xlabel('epoch', fontsize=18)
plt.grid()
plt.legend(['train', 'val'], loc='upper right')

In [ ]:
# from model.sdp import SDP 
# from service.distributeuavs import distributeUAVs
# from model.pathlossmodel import PathLossModel, NoiseMaker
# from service.combinationset import generateCombination
# UAVs = distributeUAVs()
# uavIndexes_withoutGPS = np.random.choice(env.N, env.N_ngps, replace=False)
# for id in uavIndexes_withoutGPS:
#   UAVs[id].observedPosition = None

# uavList_gps = []
# for id, uav in UAVs.items():
#   if not uav.observedPosition == None:
#     uavList_gps.append(id)

# print(uavList_gps)
# anomaly_index = np.random.choice(uavList_gps, 1, replace=False)[0]
# print(anomaly_index)

# originObservedPosition = UAVs[anomaly_index].observedPosition
# print(originObservedPosition)
# shift = np.random.choice(np.arange(500, 1000), 3, replace=False)
# print(shift)
# newObservedPosition = [originObservedPosition[0]+shift[0], originObservedPosition[1]+shift[1], originObservedPosition[2]+shift[2]]
# print(newObservedPosition)
# UAVs[anomaly_index].observedPosition = newObservedPosition
# pathLossModel = PathLossModel()
# pathLossModel.noiseMaker = NoiseMaker(mean = env.noiseMean, deviation = env.noiseDeviation)
# for i, uav_i in UAVs.items():
#   for j, uav_j in UAVs.items():
#     if not i == j:      
#       realDistance_ij = np.linalg.norm(np.array(uav_i.realPosition) - np.array(uav_j.realPosition))
#       rss = pathLossModel.estimateRSS(uav_i.power, realDistance_ij)
#       uav_i.collectRSS(j, rss)

# groupList = []
# for groupSize in range(env.DIMENSION+2, env.N_gps+1):
#   groupList.append(generateCombination(UAVs, groupSize))

# sdp = SDP()
# sdp.var_SOCPt = cp.Variable(1)
# sdp.var_SDPt = cp.Variable(1)
# sdp.var_p = [cp.Variable((env.DIMENSION+1, env.DIMENSION+1), symmetric=True) for i in range(env.N_ngps)]

# targets = []
# for uavIndex, uav in UAVs.items():
#   if uav.observedPosition is None:
#     targets.append(uav)

# results = []
# for groups in groupList:
#   for group in groups:
#     print(".", end='')
#     anchors = []
#     for i in group:
#       anchors.append(UAVs[i])

#     sdp.buildConstraint(targets, anchors, pathLossModel)
#     sdp.doSolve(log=False)
#     results.append([sdp.var_p[i].value[-1][:3] for i in range(env.N_ngps)])

# print(np.array(results).shape)
# sdpResult_uav = []
# for i in range(np.array(results).shape[1]):
#   temp = []
#   for j in range(np.array(results).shape[0]):
#     temp.append(results[j][i])
#   sdpResult_uav.append(temp)
# print(np.array(sdpResult_uav).shape)
# h = int (np.array(sdpResult_uav).shape[1])
# print(h)

# X = []
# for i in range(env.N_ngps):
#   temp = []
#   temp.append(sdpResult_uav[i][0 : h])
#   X.append(temp)
# print(np.array(X).shape)


# print(np.array(X).shape)

# new_X = []
# for i in range(np.array(X).shape[0]):
#   s = []
#   for j in range(np.array(X).shape[1]):
#     temp = []
#     for index_1, e1 in enumerate(X[i][j]):
#       for index_2, e2 in enumerate(X[i][j]):
#         if not index_1 == index_2:
#           temp.append([e1, e2])
#     s.append(temp)
#   new_X.append(s)

# X_reform = []
# rangeSample = range(np.array(new_X).shape[1])

# def swapFunc(j):
#   # print(j)
#   for sample in rangeSample:
#     temp = []
#     for i in range(np.array(X).shape[0]):
#       temp.append(new_X[i][sample][j])
#     X_reform.append(temp)

# nothing = list (map(lambda x: swapFunc(x), range(np.array(new_X).shape[2])))
# print(np.array(X_reform).shape)

# data = np.array(X_reform).reshape(len(X_reform), np.prod(np.array(X_reform).shape[1:]))
# data_min =0
# data_max = 1000
# anomaly_data = (data-data_min)/(data_max - data_min)

In [ ]:
model_mse = lambda x: np.mean(np.square(x-ae.predict(x, batch_size = batch_size)), axis=1)
fig, ax1 = plt.subplots(1,1, figsize = (8,8))
ax1.hist(model_mse(X_train), bins = np.linspace(0, 0.3, 50), label = 'Training Digits', density = True, alpha = 1.0)
ax1.hist(model_mse(y_train), bins = np.linspace(0, 0.3, 50), label = 'Testing Digits', density = True, alpha = 0.5)
ax1.hist(model_mse(anomaly_data), bins = np.linspace(0, 0.3, 50), label = 'Anomaly Digits', density = True, alpha = 0.5)
ax1.legend()
ax1.set_xlabel('Reconstruction Error');

In [ ]:
trainMean = np.mean(model_mse(X_train))
trainStd = np.std(model_mse(X_train))
testMean = np.mean(model_mse(y_train))
testStd = np.std(model_mse(y_train))
anomalyMean = np.mean(model_mse(anomaly_data))
anomalyStd = np.std(model_mse(anomaly_data))
print(trainMean,testMean,anomalyMean)
print(trainStd,testStd,anomalyStd)
# print(model_mse(X_train).shape)